# Camunda process overview example

In [ ]:
# This cell imports all required libraries and setups some helpers
import micropip
await micropip.install(["ipywidgets", "pandas", "plotly==5.1.0"])

from IPython.display import display, clear_output
from ipywidgets import widgets

import asyncio
import datetime
import js
import json
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

class Http:
    @staticmethod
    def get(url):
        request = js.XMLHttpRequest.new()
        request.open("GET", url, False)
        request.send(None)
        assert request.status in [200], request.responseText
        return json.loads(request.responseText or 'null')

## Choose process

Chosen process will be rendered below and executing the next cell will display activity data for the process.

In [ ]:
# Fetch and sort available process definiikitions
definitions = [
    (d["versionTag"] and f'{d["name"]} : {d["versionTag"]}' or f'{d["name"]} : {d["version"]:04}', d["id"])
    for d in sorted(Http.get(os.environ["CAMUNDA_ENGINE_API"] + "/process-definition"), key=lambda x: x["name"])
]

# Define BPMN display configuration
config = {"style": {"height": "400px"}}

# Define interactive widgets
definition_id = widgets.Dropdown(
     value=definitions[0][-1],
     options=definitions,
     description="Process:"
)
definition_xml = None
definition_output = widgets.Output()

# Define display init and update function
def display_bpmn(definition_id, output=definition_output, config=config):
    global definition_xml
    response = Http.get(os.environ["CAMUNDA_ENGINE_API"] + f'/process-definition/{definition_id}/xml')
    xml = definition_xml = response["bpmn20Xml"]
    with output:
        clear_output(wait=True)
        display({"application/bpmn+xml": xml, "application/bpmn+json": json.dumps(config)}, raw=True)
    
# Init display and wire display update
display(widgets.VBox([definition_id, definition_output]))
display_bpmn(definition_id.value)
definition_id.observe(lambda change: display_bpmn(change.new), 'value')

## Choose period

After choosing a different process from above, the cell below must be executed for available tasks to be updated.

In [ ]:
# Define filter widgets
started_after = widgets.DatePicker(
    value=datetime.datetime.now().date() - datetime.timedelta(14),
    description="Started:"
)

finished_before = widgets.DatePicker(
    value=datetime.datetime.now().date() + datetime.timedelta(1),
    description="Finished:"
)

selected_tasks = widgets.SelectMultiple(
   options=[],
   description="Tasks:"
)

# Define diagram placeholder
tasks_output = widgets.Output()

# Display with layout
display(widgets.VBox([widgets.HBox([started_after, finished_before, selected_tasks]), tasks_output]))

# Update task options from activity data
def set_selected_tasks_options(df):
    options = []
    names = (df[df.durationInMillis > 100].activityName.unique())
    for name in names:
        options.append((name, df[df.activityName == name].activityId.unique()[0]))                    
    selected_tasks.options = options

# Fetch data and display diagrams
def display_boxes(definition_id, selected_tasks, started_after, finished_before,
                  config=config, output=tasks_output):
    # Fetch data between chosen dates
    after = started_after.strftime("%Y-%m-%dT00:00:00.000%2B0000")
    before = finished_before.strftime("%Y-%m-%dT00:00:00.000%2B0000")
    response = Http.get(
        os.environ["CAMUNDA_ENGINE_API"] +
        f'/history/activity-instance?processDefinitionId={definition_id}' +
        f'&startedAfter={after}&finishedBefore={before}'
    )
    
    # Make pandas DataFrame
    df = pd.DataFrame(response)
    
    # Update available tasks from data
    set_selected_tasks_options(df)
    
    # Apply tasks filter and display data
    if df.empty:
        with tasks_output:
            clear_output(wait=True)
    else:
        df = df[df.durationInMillis > 100]  # filter immediate activities
        df["duration"] = pd.to_timedelta(df.durationInMillis, unit="ms")
        df["durationDT"] = pd.to_datetime(df.durationInMillis, unit="ms")

        # Create timeline graph to visualize complete processes
        timeline = px.timeline(
            df[df.activityName.notna()],
            x_start="startTime", x_end="endTime",
            y="processInstanceId",
            color="activityName",
            labels={"activityName": "Task"}
        )
        timeline.update_yaxes(visible=False, showticklabels=False)

        # Filter selected tasks only
        if selected_tasks:
            df = df[df.activityId.isin(selected_tasks)]
            
        # Create box graph for selected tasks durations
        boxes = px.box(
            df[df.endTime.notna()][["activityId", "activityName", "duration", "durationDT"]],
            x="activityName",
            y="durationDT", # Timedelta has https://github.com/plotly/plotly.py/issues/801
            color="activityName",
            labels={"durationDT": "Duration", "activityName": "Task"},
        )
        
        # display
        with tasks_output:
            clear_output(wait=True)
            display(df[["activityName", "duration"]].groupby("activityName").describe())
            display(boxes)
            display(timeline)

# Init display and wire display update
display_boxes(definition_id.value, selected_tasks.value, started_after.value, finished_before.value)  

for widget in [selected_tasks, started_after, finished_before]:
    widget.observe(lambda change: display_boxes(
        definition_id.value, selected_tasks.value, started_after.value, finished_before.value
    ), 'value')

In [ ]:
# Color selected tasks on BPMN diagram 
def color_bpmn(definition_id, selected_tasks):
    config["colors"] = {
        task_id: ({
            "stroke": "#ffffff",
            "fill": "#000000",
        } if task_id in selected_tasks else {
            "stroke": "#000000",
            "fill": "#ffffff",            
        }) for task_id in list(selected_tasks) + list(config.get("colors", []))
    }
    with definition_output:
        clear_output(wait=True)
        display({"application/bpmn+xml": definition_xml, "application/bpmn+json": json.dumps(config)}, raw=True)

selected_tasks.observe(lambda change: color_bpmn(definition_id.value, selected_tasks.value), 'value')